In [ ]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

import os

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import os
from scipy.stats import norm
import pandas as pd
from glob import glob
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [ ]:
# run params
section = "vae"
run_id = "0001"
data_name = "faces"
RUN_FOLDER = "run/{}/".format(section)
RUN_FOLDER += "_".join([run_id, data_name])


DATA_FOLDER = "/kaggle/input/celeba-dataset"
IMAGE_FOLDER = "/kaggle/input/img_align_celeba"

In [ ]:
att = pd.read_csv(os.path.join(DATA_FOLDER, "list_attr_celeba.csv"))

# imageLoader = ImageDataGenerator(IMAGE_FOLDER, INPUT_DIM[:2])

# Parámetros del modelo

In [ ]:
INPUT_DIM = (64, 64, 3)
BATCH_SIZE = 32

filenames = np.array(
    glob("../input/celeba-dataset/img_align_celeba/img_align_celeba/*.jpg")
)

NUM_IMAGES = len(filenames)

In [ ]:
EMBEDDING_DIM = 50
ENCODER_CONV_FILTERS = [32, 64, 128, 256]
DECODER_CONV_FILTERS = [256, 128, 64, 3]

# Generador para entrenar

In [ ]:
data_gen = ImageDataGenerator(rescale=1.0 / 255)

data_flow = data_gen.flow_from_directory(
    DATA_FOLDER,
    target_size=INPUT_DIM[:2],
    batch_size=BATCH_SIZE,
    shuffle=True,
    class_mode=None,
    subset="training",
)

In [ ]:
plt.imshow(data_flow.next()[0, :, :, :]);

# Generación del modelo

In [ ]:
class Sampling(tf.keras.layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [ ]:
# Encoder


def create_encoder():

    encoder_input = keras.layers.Input(shape=INPUT_DIM, name="encoder_input")

    x = encoder_input

    for i in range(len(ENCODER_CONV_FILTERS)):
        conv_layer = tf.keras.layers.Conv2D(
            filters=ENCODER_CONV_FILTERS[i],
            kernel_size=3,
            strides=2,
            padding="same",
            name="encoder_conv_" + str(i),
            activation="relu",
        )

        x = conv_layer(x)
        x = keras.layers.BatchNormalization()(x)
        # x = keras.layers.Dropout(rate = 0.25)(x)

    shape_before_flattening = K.int_shape(x)[1:]

    x = keras.layers.Flatten()(x)
    mu = keras.layers.Dense(EMBEDDING_DIM, name="mu")(x)
    log_var = keras.layers.Dense(EMBEDDING_DIM, name="log_var")(x)

    z = Sampling(name="encoder_output")([mu, log_var])

    encoder = keras.models.Model(encoder_input, [mu, log_var, z], name="encoder")
    return encoder, shape_before_flattening

In [ ]:
# Decoder
def create_decoder(shape_before_flattening):
    decoder_input = keras.layers.Input(shape=(EMBEDDING_DIM,), name="decoder_input")

    x = keras.layers.Dense(np.prod(shape_before_flattening))(decoder_input)
    x = keras.layers.Reshape(shape_before_flattening)(x)

    for i in range(len(DECODER_CONV_FILTERS)):
        conv_t_layer = keras.layers.Conv2DTranspose(
            filters=DECODER_CONV_FILTERS[i],
            kernel_size=3,
            strides=2,
            padding="same",
            name="decoder_conv_t_" + str(i),
            activation="sigmoid" if i == (len(DECODER_CONV_FILTERS) - 1) else "relu",
        )

        x = conv_t_layer(x)

        if i != (len(DECODER_CONV_FILTERS) - 1):
            x = keras.layers.BatchNormalization()(x)
            # x = keras.layers.Dropout(rate = 0.25)(x)

    decoder = keras.models.Model(decoder_input, x, name="decoder")
    return decoder

In [ ]:
class VAEModel(keras.models.Model):
    def __init__(self, encoder, decoder, r_loss_factor, **kwargs):
        super(VAEModel, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.r_loss_factor = r_loss_factor

    def compute_losses(self, data):
        z_mean, z_log_var, z = self.encoder(data)
        reconstruction = self.decoder(z)
        reconstruction_loss = tf.reduce_mean(
            tf.square(data - reconstruction), axis=[1, 2, 3]
        )
        kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
        kl_loss = tf.reduce_sum(kl_loss, axis=1)
        kl_loss *= -0.5
        total_loss = (
            self.r_loss_factor * reconstruction_loss + kl_loss
        ) / self.r_loss_factor
        return reconstruction_loss, kl_loss, total_loss

    def train_step(self, data):
        if isinstance(data, tuple):
            data = data[0]
        with tf.GradientTape() as tape:
            reconstruction_loss, kl_loss, total_loss = self.compute_losses(data)

        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        return {
            "loss": total_loss,
            "reconstruction_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }

    def call(self, inputs):
        latent = self.encoder(inputs)[2]
        return self.decoder(latent)

In [ ]:
encoder, shape_before_flattening = create_encoder()
decoder = create_decoder(shape_before_flattening)
vae = VAEModel(encoder, decoder, 10_000)

In [ ]:
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
encoder_conv_0 (Conv2D)         (None, 32, 32, 32)   896         encoder_input[0][0]              
__________________________________________________________________________________________________
batch_normalization_14 (BatchNo (None, 32, 32, 32)   128         encoder_conv_0[0][0]             
__________________________________________________________________________________________________
encoder_conv_1 (Conv2D)         (None, 16, 16, 64)   18496       batch_normalization_14[0][0]     
____________________________________________________________________________________________

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=1e-4)
vae.compile(optimizer=optimizer)
# vae.compile("adam")

# Entrenamiento

In [ ]:
def generate_data(flow):
    while True:
        flow.reset()
        while True:

            X = flow.next()
            if len(X) == BATCH_SIZE:
                yield X
            else:
                break

In [ ]:
FILEPATH = "model_checkpoint"
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    FILEPATH,
    monitor="val_loss",
    verbose=0,
    save_best_only=False,
    save_weights_only=False,
    mode="auto",
    save_freq="epoch",
)

In [ ]:
def scheduler(epoch, lr):
    return lr * tf.math.exp(-0.1)


scheduler_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
data_flow.reset()
history = vae.fit(
    generate_data(data_flow),
    steps_per_epoch=len(data_flow),
    epochs=50,
    callbacks=[checkpoint_callback, scheduler_callback],
)

Epoch 1/50
6332/6332 [==============================] - 964s 152ms/step - loss: 0.0287 - reconstruction_loss: 0.0237 - kl_loss: 50.1351
Epoch 2/50
6332/6332 [==============================] - 318s 50ms/step - loss: 0.0217 - reconstruction_loss: 0.0163 - kl_loss: 54.4973
Epoch 3/50
6332/6332 [==============================] - 301s 48ms/step - loss: 0.0209 - reconstruction_loss: 0.0153 - kl_loss: 56.2760
Epoch 4/50
6332/6332 [==============================] - 299s 47ms/step - loss: 0.0205 - reconstruction_loss: 0.0148 - kl_loss: 57.0564
Epoch 5/50
6332/6332 [==============================] - 299s 47ms/step - loss: 0.0203 - reconstruction_loss: 0.0145 - kl_loss: 57.5783
Epoch 6/50
6332/6332 [==============================] - 295s 47ms/step - loss: 0.0201 - reconstruction_loss: 0.0143 - kl_loss: 57.9293
Epoch 7/50
6332/6332 [==============================] - 298s 47ms/step - loss: 0.0200 - reconstruction_loss: 0.0142 - kl_loss: 58.1198
Epoch 8/50
6332/6332 [==============================] 

In [ ]:
import shutil

encoder.save(f"Embedding_{EMBEDDING_DIM}/encoder")
decoder.save(f"Embedding_{EMBEDDING_DIM}/decoder")
shutil.make_archive(f"Embedding_{EMBEDDING_DIM}", "zip", ".")